In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import random

In [2]:
# the basic idea is to store the data in a list of lists (one movie in a list)

In [3]:
# in this one I will try to use random user agent generator called random-user-agent v1.0.1

In [4]:
# from random_user_agent.user_agent import UserAgent
# from random_user_agent.params import SoftwareName, OperatingSystem, HardwareType

In [5]:
# software_names = [SoftwareName.CHROME.value]
# operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value,OperatingSystem.MAC.value,
#                     OperatingSystem.MAC_OS_X.value,OperatingSystem.MACOS.value]
# hardware_types = [HardwareType.COMPUTER.value]

In [6]:
# user_agent_rotator = UserAgent(software_names = software_names, 
#                                operating_systems = operating_systems,
#                                hardware_types = hardware_types,
#                                limit=100)

In [7]:
# Here will generate a list of random user of size 100

In [8]:
# user_agent = user_agent_rotator.get_user_agents()

In [9]:
# print(user_agent)

In [10]:
# the use of random_user_agent results failure data fetching due to false format

In [11]:
user_agent = [ 
    "Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html)", 
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)", 
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)", 
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)", 
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)", 
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)", 
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)", 
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)", 
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)", 
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6", 
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1", 
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0", 
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5", 
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6", 
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11", 
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20", 
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52"
] 

In [12]:
datalist = []

In [13]:
# Now I need to define a changable function that fetch one page data starting from the first one

In [14]:
def getOnePageData(Row):
    url ="https://movie.douban.com/top250?start={}&filter=".format(Row)
    res = requests.get(url,headers={'User-Agent':random.choice(user_agent)})
    return(res.text)

In [15]:
def getAllData(html):
    soup = BeautifulSoup(html, 'html.parser')
#     print(type(soup))
    movielist = soup.find('ol', class_='grid_view')
#     print(movielist)
    for movie in movielist.find_all('li'):
        one_movie_data = []
        movie_title = movie.find('span', class_ = 'title').text
        one_movie_data.append(movie_title)
        movie_score = movie.find('span', class_ = 'rating_num').text
        one_movie_data.append(movie_score)
        movie_viewer = movie.find('div', class_= 'star')
        movie_viewer_counts = re.findall(r'\d+',str(movie_viewer))[-1]
        one_movie_data.append(movie_viewer_counts)
        movie_quote = movie.find('span', class_='inq')
        if movie_quote:
            one_movie_data.append(movie_quote.text)
        else:
            one_movie_data.append('None')
        movie_link = movie.find('div',class_='hd').find('a').get('href')
        one_movie_data.append(movie_link)
        datalist.append(one_movie_data)

In [16]:
# getAllData(getOnePageData(0))
# print(datalist)

In [17]:
def save2data(fileName):
    with open(fileName, 'w') as f:
        csvfile = csv.writer(f)
        csvfile.writerow(['影片名','豆瓣评分','评论数','经典短评','豆瓣网址'])
        for movie in datalist:
            csvfile.writerow(movie)

In [18]:
def mainFunc():
    startrow = 0
    while startrow <= 225:
        print(f'We are fetching data from {startrow} to {startrow + 25}')
        html = getOnePageData(startrow)
        getAllData(html)
        startrow += 25     
    save2data('douban_top250.csv')
    print('Data fetching is over')
    return

In [19]:
mainFunc()

We are fetching data from 0 to 25
We are fetching data from 25 to 50
We are fetching data from 50 to 75
We are fetching data from 75 to 100
We are fetching data from 100 to 125
We are fetching data from 125 to 150
We are fetching data from 150 to 175
We are fetching data from 175 to 200
We are fetching data from 200 to 225
We are fetching data from 225 to 250
Data fetching is over
